# Loading packages

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Implementation of LDA

In [2]:
class LDA:
    def __init__(self, n_components):
        self.n_components = n_components
        assert self.n_components != 0, "No. of components should not be 0!"
        assert type(self.n_components) != float, "No. of components should be integer!"
        self.linear_discriminants = None
        self.mean_overall = None
        self.class_labels_count = None
        self.train_x, self.train_y = None, None

    def fit(self, X, y):
        self.train_x, self.train_y = X, y
        n_features = X.shape[1]
        class_labels = np.unique(y)
        self.class_labels_count = len(class_labels)
        assert self.n_components <= self.class_labels_count - 1, "No. of components should not exceed (no. of classes - 1) but exceeded this!"

        # Within class scatter matrix:
        # SW = sum((X_c - mean_X_c)^2 )

        # Between class scatter:
        # SB = sum( n_c * (mean_X_c - mean_overall)^2 )

        self.mean_overall = np.mean(X, axis=0)
        SW = np.zeros((n_features, n_features))
        SB = np.zeros((n_features, n_features))
        for c in class_labels:
            X_c = X[y == c]
            mean_c = np.mean(X_c, axis=0)
            # (4, n_c) * (n_c, 4) = (4,4) -> transpose
            SW += (X_c - mean_c).T.dot((X_c - mean_c))

            # (4, 1) * (1, 4) = (4,4) -> reshape
            n_c = X_c.shape[0]
            mean_diff = (mean_c - self.mean_overall).reshape(n_features, 1)
            SB += n_c * (mean_diff).dot(mean_diff.T)

        # Determine SW^-1 * SB
        A = np.linalg.inv(SW).dot(SB)
        # Get eigenvalues and eigenvectors of SW^-1 * SB
        eigenvalues, eigenvectors = np.linalg.eig(A)
        # -> eigenvector v = [:,i] column vector, transpose for easier calculations
        # sort eigenvalues high to low
        eigenvectors = eigenvectors.T
        idxs = np.argsort(abs(eigenvalues))[::-1]
        eigenvalues = eigenvalues[idxs]
        eigenvectors = eigenvectors[idxs]
        # store first n eigenvectors
        self.linear_discriminants = eigenvectors[0:self.n_components]

    def transform(self, X):
        # project data
        return np.dot(X, self.linear_discriminants.T)

    def predict(self, X):
        # predict targets

        # project the means and take their mean, y(x) = dot(X, Eigen_VECs) + threshold
        threshold = np.zeros(self.class_labels_count) # (class_count,1)
        w = 0
        for feat_mean in self.mean_overall:
            w += np.dot(self.linear_discriminants, feat_mean)
            threshold = (1/len(self.mean_overall)) * w

        threshold = threshold.reshape(-1,1)

        Y = np.zeros((len(X), self.class_labels_count))
        preds = np.zeros((len(X)), dtype=int)

        for j in range(len(X)):
            for i in range(self.n_components):
                Y[j, i] = np.dot(self.linear_discriminants[i].T,  X[j]) + threshold[i]
            preds[j] = np.argmin(Y[j])

        return preds

# Creating dataset

In [3]:
data = datasets.load_iris()

In [4]:
X = data.data
y = data.target
feats = data.target_names

In [5]:
feats

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [6]:
print(X.shape)
print(y.shape)

(150, 4)
(150,)


# Split dataset

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
test_x, test_y = X_test.copy(), y_test.copy()

In [8]:
set(y_train)

{0, 1, 2}

In [9]:
set(y_test)

{0, 1, 2}

# Applying LDA

In [10]:
lda = LDA(2)
lda.fit(X_train, y_train)

> ## Predict with LDA

In [11]:
pred = lda.predict(X_test)
pred

array([0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 2, 1, 2, 1, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 1, 2])

In [12]:
(pred == y_test).sum() / len(y_test) * 100

6.666666666666667

> ## Transform with LDA

In [13]:
X_train = lda.transform(X_train)
X_test = lda.transform(X_test)

In [14]:
X_train.shape

(120, 2)

In [15]:
X_test.shape

(30, 2)

# Using Logistic Regression

In [16]:
model = LogisticRegression(C=0.01, random_state=42)

In [17]:
model.fit(X_train, y_train)

LogisticRegression(C=0.01, random_state=42)

In [18]:
model.score(X_train, y_train)

0.9416666666666667

In [19]:
model.score(X_test, y_test)

0.9

In [20]:
f1_score(y_train, model.predict(X_train), average=None)

array([1.        , 0.90666667, 0.91764706])

In [21]:
f1_score(y_test, model.predict(X_test), average=None)

array([1.  , 0.8 , 0.88])

# Using LDA from sklearn

In [22]:
ld = LinearDiscriminantAnalysis(n_components=2, solver='svd', tol=0.5)
ld.fit(lda.train_x, lda.train_y)

LinearDiscriminantAnalysis(n_components=2, tol=0.5)

In [23]:
(ld.predict(lda.train_x) == lda.train_y).sum() / len(lda.train_y)

0.975

In [24]:
(ld.predict(test_x) == test_y).sum() / len(test_y)

0.9666666666666667